In [3]:
import pandas as pd 
preop_imaging_df = pd.read_excel('Data/Clean_imaging_data.xlsx')

Since SPECT/CT and Sestamibi are both nuclear scans, we are combining them into a single column

In [ ]:
# create a function to combine the SPECT/CT and Sestamibi 
def combine_nuclear_scans(row):
    
    # initialize the modality columns 
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']
    
    # remove missing entries
    spect_missing = pd.isnull(spect) or spect == 'No Scan'
    sestamibi_missing = pd.isnull(sestamibi) or sestamibi == 'No Scan'
    if spect_missing and sestamibi_missing:
        return 'No SPECT/CT or Sestamibi Recorded'
    
    # check if both scans are present 
    if not spect_missing and not sestamibi_missing:
        return f"SPECT/CT: {spect} | Sestamibi: {sestamibi}"
    
    # check for spect scans 
    if not spect_missing:
        return f"SPECT/CT: {spect}"
    
    # otherwise return the sestamibi scan 
    return f"Sestamibi: {sestamibi}"

In [5]:
# apply this function across the whole dataset and create the new column 
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(combine_nuclear_scans, axis=1)

# filter for only the non-localizing nuclear scans 
neg_nuclear = preop_imaging_df[preop_imaging_df['SPECT/CT-Sestamibi'].str.contains('Non-localizing')]

# create a function that checks defines localization 
def is_localizing(scan):
    if pd.isnull(scan):
        return False
    scan = scan.strip()
    return scan not in ['No Scan', 'Non-localizing']

In [8]:
# avoid the the copyright warning
neg_nuclear = neg_nuclear.copy() 

# out of the negative nuclear scans, filter for positive 4D CT
neg_nuclear['4D CT Positive'] = neg_nuclear['4D CT Scan'].apply(is_localizing)

# count total and positive cases 
total_neg_nuclear = neg_nuclear.shape[0]
num_4dct_positive = neg_nuclear['4D CT Positive'].sum()

print(f"Out of {total_neg_nuclear} nuclear-negative cases, {num_4dct_positive} had a localizing 4D CT scan.")


Out of 112 nuclear-negative cases, 54 had a localizing 4D CT scan.


**How accurate were these 4D CT scans?**

In [16]:
# filter to these 54 occurances
neg_nuclear_pos_4dct = neg_nuclear[neg_nuclear['4D CT Positive']]

# get the summary 
breakdown = neg_nuclear_pos_4dct['4D CT Scan Calculated Concordance'].value_counts()

# get percentages
breakdown_percent = breakdown / breakdown.sum() * 100

# combine count and percentage into a summary DataFrame
summary_df = pd.DataFrame({
    'Count': breakdown,
    'Percentage': breakdown_percent.round(1)
}).reset_index().rename(columns={'index': 'Concordance Type'})

print(summary_df)

  4D CT Scan Calculated Concordance  Count  Percentage
0                       Yes (Exact)     24        44.4
1            Incorrect Localization     18        33.3
2                        Yes (Side)     12        22.2
